In [ ]:
import torch
import langchain
import faiss
import re
import PyPDF2
import pypdf
from langchain_community.document_loaders import PyPDFLoader  # Example for PDF loader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [11]:
DATA_PATH = "./dataset/book.pdf"


def clean_text(page_content):
    # Step 1: Remove excessive whitespace, tabs, and newlines
    cleaned_text = re.sub(r'\s+', ' ', page_content)

    # Step 2: Remove URLs or links
    cleaned_text = re.sub(r'http[s]?://\S+|www\.\S+', '', cleaned_text)

    # Step 3: Remove unwanted special characters (keep basic punctuation)
    cleaned_text = re.sub(r'[^A-Za-z0-9\s,.!?\'"-]', '', cleaned_text)

    # Step 4: Strip leading/trailing whitespace
    cleaned_text = cleaned_text.strip()
    
    return cleaned_text


def load_documents():
    '''
        Load the documents ,
        clean the text and save the clean text in the document's page content.

        Returns: documents

    '''
    document_loader = PyPDFLoader(DATA_PATH)

    documents = document_loader.load()

    for doc in documents:
        if doc.metadata["page"] >= 645:
            doc.page_content = " "
        else:
            doc.page_content = clean_text(doc.page_content)
    return documents

def split_documents(documents):

    """
        Using simple Recursive Text Splitter we divide the document content into chunks.

        Returns: split_documents
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=100,
        length_function=len,
        is_separator_regex=False,
    )
   
    return text_splitter.split_documents(documents)


# Uncomment the code to test the code
documents = load_documents()
chunks = split_documents(documents)

In [15]:
from sentence_transformers import SentenceTransformer
import faiss  # Or chromadb

model = SentenceTransformer('google/EmbeddingGemma-300M')

ModuleNotFoundError: No module named 'numpy.rec'